In [2]:
import pandas as pd
import numpy as np
from pandas import read_csv
import glob

In [11]:
# Load all the blocks and concatenate them into one database
path = '/Users/mariabelenalberti/OneDrive - Universidad Torcuato Di Tella/Tesis/smart-meters-in-london/halfhourly_dataset' # use your path
all_files = glob.glob(path + "/*.csv")

lista = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    lista.append(df)

halfhourly_consumption = pd.concat(lista, axis=0, ignore_index=True)

ValueError: could not convert string to float: 'Null'

In [12]:
halfhourly_consumption.head()

,tstp,energy(kWh/hh)
0,2011-12-07 11:30:00,0.185
1,2011-12-07 12:00:00,0.155
2,2011-12-07 12:30:00,0.147
3,2011-12-07 13:00:00,0.164
4,2011-12-07 13:30:00,0.187


In [8]:
halfhourly_consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167817021 entries, 0 to 167817020
Data columns (total 3 columns):
LCLid             object
tstp              datetime64[ns]
energy(kWh/hh)    object
dtypes: datetime64[ns](1), object(2)
memory usage: 3.8+ GB


In [13]:
len(halfhourly_consumption)

167817021

In [9]:
del halfhourly_consumption['LCLid']

In [6]:
halfhourly_consumption['tstp'] = pd.to_datetime(halfhourly_consumption['tstp'],format = '%Y-%m-%d %H:%M')

In [25]:
halfhourly_consumption = halfhourly_consumption[halfhourly_consumption['energy(kWh/hh)'] != "Null"]

In [26]:
len(halfhourly_consumption)

167811461

In [27]:
halfhourly_consumption['energy(kWh/hh)'] = halfhourly_consumption['energy(kWh/hh)'].astype(float)

/Users/mariabelenalberti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
halfhourly_consumption['year'] = pd.DatetimeIndex(halfhourly_consumption['tstp']).year
halfhourly_consumption = halfhourly_consumption[(halfhourly_consumption['year'] == 2012) | (halfhourly_consumption['year'] == 2013)]   


/Users/mariabelenalberti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [83]:
group_by_tstp = halfhourly_consumption.groupby(['tstp','year']).mean().reset_index()

In [97]:
group_by_tstp.to_csv('group_by_tstp.csv')

In [96]:
len(group_by_tstp)

35088

In [123]:
train = group_by_tstp[:33648]
test = group_by_tstp[33648:34992]

In [124]:
len(train),len(test)

(33648, 1344)

In [125]:
train['day_of_week'] = pd.DatetimeIndex(train['tstp']).dayofweek
train['hour'] = pd.DatetimeIndex(train['tstp']).hour
train['minute'] = pd.DatetimeIndex(train['tstp']).minute

/Users/mariabelenalberti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/mariabelenalberti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mariabelenalberti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [126]:
train.head()

,tstp,year,energy(kWh/hh),day_of_week,hour,minute
0,2012-01-01 00:00:00,2012,0.230723,6,0,0
1,2012-01-01 00:30:00,2012,0.281494,6,0,30
2,2012-01-01 01:00:00,2012,0.268496,6,1,0
3,2012-01-01 01:30:00,2012,0.239779,6,1,30
4,2012-01-01 02:00:00,2012,0.228618,6,2,0


In [127]:
prediction_hh = train.groupby(['day_of_week','hour','minute']).mean().reset_index()

In [128]:
del prediction_hh['year']

In [129]:
prediction_hh

,day_of_week,hour,minute,energy(kWh/hh)
0,0,0,0,0.170981
1,0,0,30,0.178306
2,0,1,0,0.161147
3,0,1,30,0.146387
4,0,2,0,0.134911
5,0,2,30,0.127450
6,0,3,0,0.121821
7,0,3,30,0.118309
8,0,4,0,0.116975
9,0,4,30,0.119759


In [130]:
prediction_hh.to_csv('mean_prediction.csv')

In [141]:
folder = "/Users/mariabelenalberti/OneDrive - Universidad Torcuato Di Tella/Tesis/code_datasets/2_media/"
file = "y_pred.csv"
y_pred = pd.read_csv(folder+file)

In [142]:
y_pred = y_pred['energy(kWh/hh)']

In [147]:
y_pred.to_list()

[0.17098122899999998,
 0.178305988,
 0.161147126,
 0.14638737599999999,
 0.134911211,
 0.12745016,
 0.121820863,
 0.118308825,
 0.11697548199999999,
 0.119758882,
 0.12483715,
 0.137688856,
 0.155485964,
 0.179554438,
 0.19927038600000002,
 0.209520675,
 0.216011686,
 0.217161252,
 0.213081513,
 0.21020637600000003,
 0.208645508,
 0.20699385,
 0.205767315,
 0.20759756899999998,
 0.20898454800000002,
 0.210244181,
 0.208368716,
 0.20599661100000002,
 0.20268017,
 0.203436315,
 0.205853206,
 0.21357694300000002,
 0.22610923600000002,
 0.24560450399999997,
 0.266459705,
 0.28948344800000003,
 0.305738589,
 0.32094994,
 0.32653620899999997,
 0.328506805,
 0.325888255,
 0.319317442,
 0.309435481,
 0.295501009,
 0.27007843600000003,
 0.24253044899999998,
 0.210888646,
 0.181996397,
 0.169576502,
 0.177668297,
 0.160096988,
 0.146058244,
 0.13429740099999998,
 0.126780766,
 0.121684371,
 0.118098163,
 0.117181928,
 0.120224235,
 0.12604390699999998,
 0.140328607,
 0.159574471,
 0.183745295,
 

In [145]:
y_test = test['energy(kWh/hh)']

In [146]:
y_test.to_list()

[0.2161182436878046,
 0.1901067733554552,
 0.16506753855162978,
 0.1474635956080422,
 0.13550146406402475,
 0.12999199691586935,
 0.12290201054069858,
 0.11994046461057986,
 0.11756353695100505,
 0.115917431075541,
 0.11826742147179337,
 0.12613937144251405,
 0.13665547538551626,
 0.16111985176654323,
 0.1905112239703304,
 0.2280782743704857,
 0.24593908623584484,
 0.24526454529480748,
 0.23469718863725048,
 0.2233234432363848,
 0.22268690218622017,
 0.22095978149160583,
 0.21443010536899745,
 0.20909683691917208,
 0.21486353006638045,
 0.2262010932448263,
 0.2229033582780155,
 0.21863627497071497,
 0.2192657166146029,
 0.22242951967981173,
 0.22177333071065916,
 0.23116793591095458,
 0.25106207296505983,
 0.290565599414291,
 0.3204925810425615,
 0.348786648116338,
 0.3633956663868833,
 0.37333476449346115,
 0.3805387466718729,
 0.3783078726313736,
 0.3709105990240096,
 0.3676011036854551,
 0.35004707027343773,
 0.3379707028125009,
 0.3169515624023435,
 0.2968667967968751,
 0.264337173

In [150]:
from sklearn import metrics

In [151]:
metrics.mean_absolute_error(y_test,y_pred)

0.03366915340962402